<font size="6">Нейронные сети</font>

# Ограничения линейных моделей

Вспомним материал лекции №2, на которой мы разбирали линейные модели. Напомним, что выход линейной модели является линейной комбинацией входных признаков, к которой также добавляется смещение:

$$ \large
\begin{eqnarray*}
y & = & w_1 x_1+ w_2 x_2 + ... + w_n x_n + b\\
& = & \sum_{i=1}^n x_i w_i + b \\
& = & (\vec{x}, \vec{w}) + b
\end{eqnarray*}
$$

Данную формулу можно графически представить следующим образом:

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/one_neuron_linear_model.png" width="450"></center>

Если линейная модель имеет множество выходов, то каждый выход $\large y_j$ имеет свой собственный вектор весов $\large \vec{w_j}$ и свое смещение $\large b_j$ :

$$ \large y_j = (\vec{x}, \vec{w_j}) + b_j \ \ \ \ \ \ \ j= 1 ... C,$$

где $\large \vec{w_j}=[w_{1j}, w_{2j}, ... w_{nj}]^\top$.

Тогда выход всей модели можно записать в векторно-матричном виде:

$$\large \underset{[1 \times C]}{\vec{y}} = \underset{[1 \times n]}{x} \underset{[n \times C]}{W} + \underset{[1 \times C]}{\vec{b}}$$

Здесь матрица весовых коэффициентов $\large W$ образуется путем объединения векторов весовых коэффициентов для каждого выхода, а вектор смещений $\large \vec{b}$ — соответственно путем объединения смещений всех выходов:

$$ \large
\begin{eqnarray*}
W & = & [\vec{w_1}, \vec{w_2}, ... \vec{w_C}]\\
\vec{b} & = & [b_1, b_2, ... b_c]
\end{eqnarray*}
$$

Графически это можно представить следующим образом:

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/linear_classifier.png" width="450"></center>

Вспомним рассмотренный в лекции №2 датасет из статьи [[paper] 🎓 Emotion classification in Russian: feature engineering and analysis](https://link.springer.com/chapter/10.1007/978-3-030-72610-2_10).

Он размечен по **5 классам эмоций**:
- радость (joy),
- печаль (sadness),
- злость (anger),
- неуверенность (uncertainty),
- нейтральность (neutrality).

Рассмотрим линейный классификатор, обученный на этом датасете:

- Метод векторизации: TF-IDF
- Модель: логистическая регрессия
- Усредненное значение accuracy: $0.84$.


В линейном классификаторе каждый выход модели отвечает за свой класс. Мы можем надеяться, что в процессе обучения линейного классификатора весовые вектора разных выходов выучат некий обобщенный "шаблон" образцов данного класса. Тогда при подаче на вход нового образца один из шаблонов будет "подходить" больше других, и соответствующий выход будет получать **наибольшее значение взвешенной суммы**. Номер выхода, который имеет наибольшее значение взвешенной суммы, будет восприниматься как предсказанный класс.

Визуализируем шаблоны (векторы весовых коэффициентов), получающиеся в результате обучения линейного классификатора на датасете, размеченном по классам эмоций.

*КАРТИНКА linear_classifier_emotions.png*

*Исходник: EduNet-content/dev-2.1/L05/out/linear_classifier_mnist.png*

*В нем нужно заменить цифры на смайлики из картинки EduNet_NLP-content/L01/out/sentiment_task.png*

*Например, 0 заменить на зеленый смайлик, а 9 заменить на розовый смайлик*

*Если это возможно и не займет много времени, можно цвет стрелочек сделать таким же, как цвет смайликов: синие стрелочки заменить на зеленые, оранжевые стрелочки заменить на розовые*

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L05/out/linear_classifier_mnist.png" width="450"></center>

Для каждого класса были посчитаны свои коэффициенты регрессии. Они представляют матрицу $5 \times 1739$, где $5$ — количество классов, $1739$ — количество признаков (слов).

Ожидается, что наиболее характерным слова для каждой эмоции будут присвоены самые большие коэффициенты соответствующего класса.

Выведем значения коэффициентов для слов, которые предположительно наиболее характеры для каждого класса эмоции. Также выведем топ-5 слов с самыми большими коэффициентами.

In [ ]:
!wget -q https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L03/coefficient_matrix.txt
!wget -q https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L03/feature_names.txt

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

W = np.loadtxt("coefficient_matrix.txt") # load coefficients, shape (5, 1739)
X = np.loadtxt("feature_names.txt", dtype=object) # load features dictionary

labels = ['Злость', 'Радость', 'Нейтральность',
          'Печаль', 'Неуверенность'] # class labels in dataset
expected = ['злой', 'рад', 'нормальный',
            'грустный', 'думать'] # expected typical words for each class
f, axs = plt.subplots(1, 5)

for i in range(W.shape[0]):

    order = W[i].argsort() # ascending order for coefficients
    top_Wi = W[i][order][::-1][:5] # sort and extract top-5 coefficients
    top_Wi = np.insert(top_Wi,0,
                       W[i][np.where(X == expected[i])]) # insert coefficient for expected word
    top_X = X[order][::-1][:5] # words with the highest coefficients
    top_X = np.insert(top_X, 0, expected[i]) # insert expected word
    barlist = axs[i].bar(x=[x[0] for x in zip(top_X, top_Wi)],
         height=[x[1] for x in zip(top_X, top_Wi)])
    axs[i].tick_params(labelrotation=65)
    axs[i].title.set_text(labels[i])
    barlist[0].set_color('g')

f.set_figheight(3)
f.set_figwidth(20)
plt.show()

Можно заметить, что наши предположения не подтвердились: не все слова, характерные для какой-либо эмоции, получают высокий коэффициент. Следовательно, модель не учитывает вариативность данных внутри этого класса.

Ограничение линейного классификатора состоит в том, что для каждого класса существует только один шаблон. Шаблон каждого класса будет пытаться вобрать в себя информацию обо всех объектах класса сразу. Сильная внутриклассовая вариативность будет мешать линейному классификатору запоминать разные варианты объектов одного класса, и это ограничивает точность модели.

##  ХОR — проблема

На самом деле линейный классификатор плохо справляется даже с более простыми задачами, чем классификация изображений. Рассмотрим такую задачу: на вход модели подаётся упорядоченный набор из двух чисел согласно таблице истинности некоторой логической функции. Задача линейного классификатора — сопоставить этим числам их класс согласно таблице. Графически два входных числа можно изобразить как координаты точек на плоскости, а знаком $+/-$ обозначить их истинный класс. Тогда задача классификатора — построить линию, отделяющую "плюсы" (класс 1) от "минусов" (класс 0).

В случае моделирования логических функций "И" и "ИЛИ" сложностей не возникает: в пространстве признаков без труда можно провести прямую линию, разделяющую точки разных классов. В случае моделирования логической функции "исключающее ИЛИ" (XOR) видно, что одной линией разделить точки разных классов геометрически невозможно — точки, размеченные по таблице истинности XOR, являются **линейно неразделимыми**.

То есть линейный классификатор уже не может справиться с этой, казалось бы, простой задачей.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/xor_problem.png" width="1000"></center>

# Многослойные нейронные сети

Одна из идей для решения проблемы внутриклассовой вариативности — модифицировать модель таким образом, чтобы у нее внутри было не по одному шаблону на каждый класс, а по несколько (скажем, 100 шаблонов вместо 10 при десяти классах). Тогда бы модель имела возможность запоминать разные объекты одного класса и далее использовать эти промежуточные шаблоны для разбиения объектов на классы.

*КАРТИНКА modeified_model_nlp.png*

*Исходник: EduNet-content/dev-2.1/L05/out/modified_model.png*

*Картинки животных нужно заменить на тексты:*

- *Лошадь слева → Меня **разочаровал** его поступок.*
- *Лошадь справа → Я **недоволен** оценкой за экзамен.*
- *Кот → Сегодня **замечательный** день.*

*Слова, выделенные жирным, выделены цветом: **разочаровал** и **недоволен** — розовым цветом, **замечательный** — зеленым цветом*

*Тексты могут быть просто в прямоугольниках либо в "пузырях" для диалога ([пример 1](https://cdn.icon-icons.com/icons2/38/PNG/512/dialoguebubble_4874.png), [пример 2](https://w7.pngwing.com/pngs/389/478/png-transparent-callout-speech-balloon-callout-miscellaneous-angle-white-thumbnail.png))*

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L05/out/modified_model.png"  width="600"></center>

Реализуем эту модель на основе линейного классификатора из лекции №2.

Применяем к выходам классификатора еще один классификатор. Будет ли данная модель работать лучше?

In [ ]:
import numpy as np


x = np.random.rand(1739)  # random text
W1 = np.random.randn(1739, 100) * 0.0001  # without bias
W2 = np.random.randn(100, 10) * 0.0001  # without bias

scores1 = np.matmul(x, W1)  # matrix multiplication, equivalent x @ W1
scores2 = np.matmul(scores1, W2)  # matrix multiplication, of the next classifier

print(f"First classifier shape: {scores1.shape}")
print(f"Second classifier shape: {scores2.shape}")

Нетрудно заметить, что последовательное применение двух классификаторов к входным данным эквивалентно применению одного классификатора с матрицей весов, равной произведению двух матриц весов классификаторов, примененных последовательно.

$$\large \text{scores}_1 = x \cdot W_1 $$

$$\large \text{scores}_2 = \text{scores}_1 \cdot W_2 = x  \cdot W_1 \cdot W_2 $$

$$\large W = W_1 \cdot W_2 $$

$$\large \text{scores}_2 = x \cdot W $$

Для того, чтобы последовательно примененные классификаторы не вырождались в один, необходимо применить нелинейность к их выходам. Например, пропустить результат применения первой матрицы весов через сигмоиду:

$$\large \sigma(s)=\frac{1}{1+e^{-s}}$$

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/sigmoid_function.png" width="500"></center>

In [ ]:
def sigmoid(s):
    return 1 / (1 + np.exp(-s))


# define vectorized sigmoid to implement with ndarray element-wise
sigmoid_np = np.vectorize(sigmoid)

scores1 = np.matmul(x, W1)
activations = sigmoid_np(scores1)  # values after non-linear function
scores2 = np.matmul(activations, W2)

print(f"First classifier shape: {scores1.shape}")
print(f"Activations shape: {scores1.shape}")
print(f"Second classifier shape: {scores2.shape}")

Теперь вычисления выглядят так:

$$\large \text{scores}_1 = x \cdot W_1 $$

$$\large \text{activations} = σ(\text{scores}_1) $$

$$\large \text{scores}_2 = \text{activations} \cdot W_2 =σ(x \cdot W_1) \cdot W_2$$

Такая нелинейная функция, примененная к результату линейного преобразования, называется **функцией активации**. И мы уже пользовались подобной, когда разбирали Cross-Entropy Loss (Softmax).

Приведем код в порядок:

In [ ]:
class NeuralNet:
    def __init__(self):
        self.W1 = np.random.randn(3072, 100) * 0.0001
        self.W2 = np.random.randn(100, 10) * 0.0001

    def predict(self, x):
        scores1 = np.matmul(x, self.W1)  # Linear
        activations = sigmoid_np(scores1)  # activation Sigmoid
        scores2 = np.matmul(activations, self.W2)  # Linear

        return scores2


x = np.random.rand(3072)  # image
model = NeuralNet()
scores = model.predict(x)
print(f"Model output shape: {scores.shape}")

Ядром вычислений по-прежнему является скалярное произведение входов с весовыми коэффициентами.

Таким образом вводится модель искусственного нейрона — базового элемента искусственной нейронной сети. Выходом нейрона является результат применения функции активации к взвешенной сумме входных сигналов (в общем случае с учетом смещения — "bias").


<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/neurons_output.png" width="900"></center>

Нейроны в нейронных сетях объединяют в слои и соединяют между слоями по принципу "каждый с каждым". Так получаются **многослойные полносвязные нейронные сети** (fully-connected networks). Синонимичным названием является **многослойный персептрон**.

Пример многослойного персептрона с двумя скрытыми слоями:

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/nn_fully_connected.png"  width="500"></center>

Как добавление в модель скрытых слоев с нелинейностями позволяет решать линейно неразделимые задачи (вроде XOR или более сложные) можно пронаблюдать [в интерактивном тренажере от TensorFlow 🎮[demo]](http://playground.tensorflow.org/#activation=linear&batchSize=10&dataset=xor&regDataset=reg-plane&learningRate=0.1&regularizationRate=0&noise=0&networkShape=&seed=0.62952&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false).

##  Веса и смещения

Веса и смещения являются настраиваемыми параметрами в нейронной сети. Они настраиваются в процессе обучения, поэтому их также называют **обучаемыми параметрами**. Рассмотрим, как различные значения обучаемых параметров влияют на работу одного нейрона. Это поможет нам получить интуицию, как добавление в модель скрытых слоев с нелинейностями наделяет её большей выразительной способностью.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/why_add_bias_example.png" width="500"></center>

Рассмотрим простой пример. На вход нейрона подаётся входное значение $x$, умноженное на вес $w$. После применения сигмоидальной функции активации, в зависимости от веса, при всевозможных значениях входа мы можем получить следующие графики при $w$, равном $0.5$, $1$ и $2$:

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/sigmoid_with_different_weights.png" width="700"></center>

Можно заметить, что значение веса меняет **крутизну** итоговой сигмоидальной функции.

Но что, если требуется, чтобы при $x=5$ нейрон выдавал $0$? Изменением крутизны сигмоиды этого не добиться. Требуется дополнительный параметр — смещение.


**Смещение** (англ. bias) — это дополнительный коэффициент, прибавляющийся к взвешенной сумме входов. Наличие смещения позволяет сдвинуть функцию активации влево или вправо.



<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/add_bias_example.png" width="500"></center>

Тогда при смещениях, равных $5$, $0$ и $-5$, мы можем получить сдвинутые функции активации, что способствует лучшему обучению нейронной сети:

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/sigmoid_with_different_biases.png" width="700"></center>

## Библиотека PyTorch

Практически вся наша работа c нейронными сетями будет осуществляться с помощью [PyTorch 🛠️[doc]](https://pytorch.org/), поэтому необходимо познакомиться с основными концептами, принципами и функциями этой библиотеки.

Лучший друг в этом, конечно же, [документация 🛠️[doc]](https://pytorch.org/docs/stable/index.html), здесь же мы разберем только основные сущности и методы.

Класс `torch.Tensor` [🛠️[doc]](https://pytorch.org/docs/stable/tensors.html#torch.Tensor) предоставляет функциональность работы с многомерными массивами.

Создание "пустого" тензора:

In [ ]:
import torch

a = torch.Tensor()

Функция создания тензора из списка:

In [ ]:
a = torch.tensor([1.1, 2.2, 3.2])
a.dtype

Явное указание типа данных:

In [ ]:
a = torch.tensor([1.1, 2.2, 3.2], dtype=torch.float64)
a.dtype

Создание двумерного тензора, заполненного единицами (для нулей `zeros`)

In [ ]:
a = torch.ones(size=(3, 2))
a.size()

Создание двумерного тензора, заполненного указанным значением


In [ ]:
a = torch.full(size=(3, 2), fill_value=3.74)
a

Транспонирование (изменение порядка осей)

In [ ]:
a = a.T
a

В библиотеке реализовано большое количество математических функций

In [ ]:
c = torch.exp(a)
print("Exponents tensor:\n", c)

c += 1
print("\nAdd 1 to tensor:\n", c)

Почти всё, что есть в NumPy, есть в PyTorch. Например, суммирование значений тензора с помощью `.sum()`:

In [ ]:
c.sum()

Перестановка, удаление и добавление пространственных измерений:

In [ ]:
a = torch.zeros((2, 5, 1, 8))
print("Original tensor size:\n", a.size())

a = a.permute(dims=(2, 0, 3, 1))  # permute dimensions
print("After permute tensor size:\n", a.size())

a = a.squeeze()  # delete dimension
print("After squzee tensor size:\n", a.size())

a = a.unsqueeze(dim=0)  # add dimension
print("After unsquzee tensor size:\n", a.size())

Преобразование `torch.Tensor` в `np.ndarray`:

In [ ]:
a.numpy()

PyTorch позволяет тензору быть представлением (view) существующего тензора. Тензор представления использует те же данные, что и его базовый тензор. Поддержка `view` позволяет избежать явного копирования данных, что позволяет экономить память.

In [ ]:
a = torch.rand(2, 8)
print("Original tensor:\n", a)

b = a.view(4, 4)
print("\nTensor after view:\n", b)

print("\nTensor b uses the same memory space as tensor a:")
id(a[0, 0]) == id(b[0, 0])

Размещение тензора на GPU:

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Cuda available: {torch.cuda.is_available()} \n")

a = a.to(device)  # moving tensor to gpu
b = torch.full_like(a, 2).to(device)
c = a * b  # compute on gpu (more fast with parallel computing)
c

## Обучение нейронной сети

Процесс обучения нейронных сетей состоит из двух этапов: прямого и обратного распространения.

Во время **прямого распространения** (forward pass) мы подаем на вход нейронной сети пример данных, сигналы проходят через наши слои, на каждом слое применяется **функция активации**. В конце производится расчет значений на выходе модели $y_{pred}$, которые передаются в функцию потерь $\text{Loss}$ для сравнения с целевыми значениями $y_{true}$.

$$\large y_{pred}=\text{model}(x, 𝐖)$$

$$\large L=\text{Loss}(y_{true}, y_{pred})$$

Значение функции потерь зависит от целевых значений $y_{true}$, входных данных $x$ и параметров модели $𝐖$.

$$\large L=\text{Loss}(y_{true}, \text{model}(x, 𝐖)) = f(y_{true}, x, 𝐖)$$

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/forward_pass.png" width="450"></center>

На этапе **обратного распространения (backward propagation)** осуществляется подсчет **градиента** функции потерь о обучаемым параметрам $\nabla_𝐖L$. Это необходимо для **обновления** параметров модели.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/backward_pass.png" width="450"></center>

В рамках этой лекции мы подробно рассмотрим этапы прямого распространения (применение функции активации и функции потерь) и обратного распространения (метод обратного распространения ошибки).

#  Функции активации

Идея применения функций активации в структуре искусственных нейронных сетей обусловлена биологической аналогией. Известно, что в биологических нейронных сетях имеется аналог нелинейной функции активации: существует пороговый потенциал, только после достижения которого происходит возбуждение (активация) нейрона и, как следствие, распространение сигнала далее по нейронной сети.

Именно таким простейшим образом ведёт себя пороговая функция активации, которая использовалась при построении первых искусственных нейронных сетей — перцептронов:

$$\large f(x) =
\begin{cases}
0, &\text{$x<b$} \\
1, &\text{$x\geq b$}
\end{cases}
$$

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/threshold_function_plot.png" width="300"></center>

##  Свойства функций активации

Функции активации должны обладать следующими свойствами:

* **Нелинейность:** функция активации необходима для введения нелинейности в нейронные сети. Если функция активации не применяется, выходной сигнал становится простой линейной функцией. Нейронная сеть $\text{NN}$ без нелинейностей будет действовать как линейная модель с ограниченной способностью к обучению:
$$\large \hat{y}=\text{NN}(X,W_1,...,W_n)=X\cdot W_1\cdot ...\cdot W_n=X\cdot W$$
Только нелинейные функции активации позволяют нейронным сетям решать задачи аппроксимации нелинейных функций:
$$\large \hat{y}=\text{NN}(X,W_1,...,W_n)=\sigma(...\sigma(X\cdot W_1)...\cdot W_n)\neq X\cdot W$$

* **Дифференцируемость:** функции активации должны быть способными пропускать градиент, чтобы было возможно оптимизировать параметры сети градиентными методами, в частности использовать алгоритм обратного распространения ошибки.

Рассмотрим наиболее популярные функции активации и обсудим их преимущества и недостатки.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L03/popular_activation_functions.png" width="700"></center>

<center><em>Источник: <a href="https://arxiv.org/pdf/1911.05187.pdf">AI in Pursuit of Happiness, Finding Only Sadness: Multi-Modal Facial Emotion Recognition Challenge</a></em></center>

## Логистическая функция

Логистическая (сигмоидальная) функция — используется в задачах бинарной классификации, в основном после выхода последнего нейрона. Позволяет определить вероятность принадлежности к одному из двух классов (0 или 1).

$$\large \sigma(x)=\frac{1}{1+e^{-x}}$$

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/sigmoid_function.png" width="500"></center>

В отличие от пороговой функции активации, где у нейрона было всего два состояния: "активирован" или "не активирован", с логистической функцией для нейрона возможны значения "активирован на $50\%$", "активирован на $20\%$" и так далее. Если активированы несколько нейронов, можно найти нейрон с наибольшим значением активации.

[[doc] 🛠️ Сигмоидальная функция активации в PyTorch](https://pytorch.org/docs/stable/generated/torch.nn.Sigmoid.html):
```python
torch.nn.Sigmoid()
```

In [ ]:
import torch
from torch import nn

activation = nn.Sigmoid()
input_values = torch.randn(5) * 5
activation_sig = activation(input_values)
print(f"input_values: {input_values}\nactivation_sig: {activation_sig}")

Значения логистической функции активации лежат в интервале от 0 до 1, что делает ее полезной для моделирования вероятностных величин. Значения сигмоиды можно интерпретировать как вероятности, что делает ее полезной в задачах бинарной, а также многозначной (multilabel) классификации.

##  Гиперболический тангенс

Гиперболический тангенс схож с логистической функцией. Он определяется следующей формулой:

$$\large \tanh(x)=\frac{e^x - e^{-x}}{e^x+e^{-x}}$$

*КАРТИНКА tanh_function.png*

*Исходник: EduNet-content/dev-2.1/L05/out/activation_function_tanh.png*

*Нужно оставить только левый график tanh function, а правый график tanh derivative убрать*

*Должно получиться аналогично картинке sigmoid_function.png*

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L05/out/activation_function_tanh.png" width="1000"></center>

Значения гиперболического тангенса лежат в интервале от $-1$ до $1$, что обеспечивает симметричность относительно нуля. Гиперболический тангенс применяют в основном в скрытых слоях нейронных сетей для моделирования нелинейных зависимостей в данных.

[[doc] 🛠️ Гиперболический тангенс в PyTorch](https://pytorch.org/docs/stable/generated/torch.nn.Tanh.html):
```python
torch.nn.Tanh()
```

In [ ]:
activation = nn.Tanh()
input_values = torch.tensor([11.1529, 4.3029, 0.5081, -3.8456, -1.9058])
activation_tanh = activation(input_values)
print(f"input_values: {input_values}\nactivation_tanh: {activation_tanh}")

##  ReLU

Часто на практике применяется функция активации ReLU. Значение данной функции равно нулю для всех отрицательных входных значений и равно входному значению, если оно неотрицательно. Название ReLU (Rectified Linear Unit), "выпрямитель", связано с электротехнической аналогией.

$$\large \text{ReLU}(x)=\max(0,x)$$

*КАРТИНКА relu_function.png*

*Исходник: EduNet-content/dev-2.1/L05/out/activation_function_relu.png*

*Нужно оставить только левый график ReLU function, а правый график ReLU derivative убрать*

*Должно получиться аналогично картинке sigmoid_function.png*

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L05/out/activation_function_relu.png" width="1000"></center>

[[doc] 🛠️ ReLU в PyTorch](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html):
```python
torch.nn.ReLU()
```

In [ ]:
activation = nn.ReLU()
input_values = torch.randn(5)
activation_relu = activation(input_values)
print(f"input_values: {input_values}\nactivation_relu: {activation_relu}")

Значения ReLU лежат в диапазоне $[0, + \infty)$. Все отрицательные значения обнуляются, а положительные значения остаются без изменений. Нейроны с отрицательными входами имеют нулевой градиент и могут остановить обновление своих весов.

Функцию активации ReLU применяют в нейронных сетях в основном в скрытых слоях. Она широко используется во многих архитектурах, доказавших свою эффективность на практике.

## Leaky ReLU

Leaky ReLU (ReLU с «утечкой», название также обусловлено электротехнической аналогией) является простейшей модификацией описанной выше ReLU, призванной исправить проблему "умирания" отдельных нейронов. В отличие от ReLU, данная функция не равна константе $0$ при всех отрицательных входных значениях, а реализует в этой области линейную зависимость с небольшим угловым коэффициентом (например, с угловым коэффициентом $10^{-2}$).

$$\large \text{LeakyReLU}(x, \alpha)=\max(\alpha x,x), \ \ \ \alpha<1$$

*КАРТИНКА leaky_relu_function.png*

*Исходник: EduNet-content/dev-2.1/L05/out/activation_function_leaky_relu.png*

*Нужно оставить только левый график LeakyReLU function, а правый график LeakyReLU derivative убрать*

*Должно получиться аналогично картинке sigmoid_function.png*

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L05/out/activation_function_leaky_relu.png" width="1000"></center>

[[doc] 🛠️ Leaky ReLU в PyTorch](https://pytorch.org/docs/stable/generated/torch.nn.LeakyReLU.html):
```python
torch.nn.LeakyReLU()
```

In [ ]:
activation = nn.LeakyReLU(0.01)
input_values = torch.randn(5)
activation_lrelu = activation(input_values)
print(f"input_values: {input_values}\nactivation_lrelu: {activation_lrelu}")

Значения LeakyReLU лежат в диапазоне $(-\infty, +\infty)$. Использование ненулевого наклона для отрицательных значений помогает избежать проблемы "умерших нейронов", которая возникает при использовании обычной ReLU. Однако необходимо выбирать значение для наклона отрицательной части, что может потребовать дополнительного подбора и настройки.

Функцию активации LeakyReLU применяют в нейронных сетях в основном в скрытых слоях для предотвращения проблемы "умерших" нейронов, особенно в глубоких архитектурах.

##  GELU (Gaussian Error Linear Unit)

Функция активации, используемая в трансформерах: Google BERT и OpenAI GPT-2.

$$\large \text{GELU}(x)=xP(X\leq x)=x\Phi(x)=x\cdot \frac{1}{2}[1+erf(\frac{x}{\sqrt{2}})]$$

$$\large erf(x)=\frac{2}{\sqrt{\pi}}\int_0^xe^{-t^2}dt$$


На практике GELU может быть приблизительно вычислена так:
$$\large \text{GELU}(x)\approx 0.5x(1+\tanh[\sqrt{2/\pi}(x+0.044715x^3)])$$

или

$$\large \text{GELU}(x) \approx x\cdot \sigma(1.702x)$$

*КАРТИНКА gelu_function.png*

*Исходник: EduNet-content/dev-2.1/L05/out/activation_function_gelu.png*

*Нужно оставить только левый график GELU function, а правый график GELU derivative убрать*

*Должно получиться аналогично картинке sigmoid_function.png*

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L05/out/activation_function_gelu.png" width="1000"></center>

[[doc] 🛠️ GELU в PyTorch](https://pytorch.org/docs/stable/generated/torch.nn.GELU.html):
```python
torch.nn.GELU()
```

In [ ]:
activation = nn.GELU()
input_values = torch.randn(5) * 5
activation_gelu = activation(input_values)
print(f"input_values: {input_values}\nactivation_gelu: {activation_gelu}")

#  Функции потерь (loss functions)

Предположим, у нас есть нейронная сеть с некоторыми весами. Прежде всего мы должны понять, насколько она точна, то есть насколько наши ожидания соответствуют результату работы нейронной сети. Мы подали на вход нейронной сети пример данных, сигналы прошли через наши слои и функции активации вперёд **(forward propagation)**, и на выходе мы имеем предсказание. Как оценить его точность?

Для оценки соответствия полученного результата ожидаемому используют функцию потерь (loss function). Значение функции потерь даёт количественную оценку величины такого соответствия.

Функция потерь в нейронной сети принимает два аргумента:
* вектор значений, который мы считаем априорно верным;
* вектор значений конечных выходов модели, который должен соответствовать априорно верному.

[[doc] 🛠️ Функции потерь в PyTorch](https://pytorch.org/docs/stable/nn.html#loss-functions)

###  Cross-Entropy

Кросс-энтропия — классическая функция потерь **при решении задач классификации**. Возникновение такой формы измерения различия между целевой переменной и предсказанием модели в задаче классификации мы уже рассматривали на лекции про линейные классификаторы. Напомним здесь формулу, по которой она рассчитывается.

Для $i$-го объекта выборки, если выходной вектор состоит из $C$ компонент (**логитов** для $C$ классов), кросс-энтропия между выходом модели $\hat{y}$ и целевым вектором $y$ будет равна

$$\large \text{CE}_i(\hat{y},y)= - \sum_{k=1}^{C}{y_{ik}\cdot\log\left(\frac{\exp(\hat{y}_{ik})}{\sum_{j=1}^{C}\exp(\hat{y}_{ij})}\right)}$$

При вычислении по всему набору данных (или по мини-батчу) из $N$ объектов ошибка на отдельных объектах усредняется:

$$\large \text{CE}=\frac{1}{N}\sum_{i=1}^{N}\text{CE}_i$$


[[doc] 🛠️ Cross-Entropy Loss в PyTorch](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss):
```python
torch.nn.CrossEntropyLoss()
```

Обратите внимание, что Cross-Entropy Loss в PyTorch уже включает в себя Softmax и принимает в качестве выхода модели логиты. Поэтому при использовании данной фукции потерь ставить на последнем слое нейронной сети Softmax **не нужно**.

In [ ]:
import torch
from torch import nn

criterion = nn.CrossEntropyLoss()


# fmt: off
model_output = torch.tensor([[2.4, 1.9, 7.3],
                             [9.5, 2.7, 4.0],
                             [5.7, 4.1, 0.2]])  # logits
# fmt: on

print(f"model_output:\n {model_output}")

target = torch.tensor([2, 0, 1], dtype=torch.long)  # class labels
print(f"target: {target}")

loss_ce = criterion(model_output, target)
print(f"loss_ce: {loss_ce}")

$$\text{CE}_1 = - \log\left(\frac{\exp{(7.3)}}{\exp{(2.4)}+\exp{(1.9)}+\exp{(7.3)}}\right)$$

$$\text{CE}_2 = - \log\left(\frac{\exp{(9.5)}}{\exp{(9.5)}+\exp{(2.7)}+\exp{(4.0)}}\right)$$

$$\text{CE}_3 = - \log\left(\frac{\exp{(4.1)}}{\exp{(5.7)}+\exp{(4.1)}+\exp{(0.2)}}\right)$$

$$\text{CE} = \frac{1}{3}(\text{CE}_1 + \text{CE}_2 + \text{CE}_3)$$

In [ ]:
import numpy as np

ce_1 = -np.log(np.exp(7.3) / (np.exp(2.4) + np.exp(1.9) + np.exp(7.3)))
ce_2 = -np.log(np.exp(9.5) / (np.exp(9.5) + np.exp(2.7) + np.exp(4.0)))
ce_3 = -np.log(np.exp(4.1) / (np.exp(5.7) + np.exp(4.1) + np.exp(0.2)))

ce = (1 / 3) * (ce_1 + ce_2 + ce_3)
print(f"hand-calculated loss_ce: {ce}")

* **Преимущества:** Важное свойство кросс-энтропии — возможность работать с весами для классов, а значит и возможность применения этой функции потерь при работе с несбалансированным датасетом.

#### Веса классов

При работе с несбалансированными классами модель будет чаще сталкиваться с доминирующим классом и вместо того, чтобы разбираться в признаках объектов, может начать ориентироваться на статистическое распределение классов.

Пример: датасет, в котором $95\%$ объектов относятся к классу $1$ и $5\%$ — к классу $0$. Модель может выучиться всегда относить объекты к классу $1$, и в $95\%$ случаях она будет права.

В случае дисбаланса классов в задаче классификации можно изменить функцию потерь так, чтобы больше штрафовать модель за ошибки в минорных классах.

В `CrossEntropyLoss` [🛠️[doc]](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) в PyTorch  есть параметр `weight`, который имеет по умолчанию значение `None`. В него можно передать тензор весов **размером с количество классов** и получить взвешенную функцию потерь.

Посмотрим, как это работает. Допустим, мы получили от нейросети неверные предсказания: второй объект должен относиться к классу $1$, а не $0$:

In [ ]:
# fmt: off
# Scores for batch of two samples
model_output = torch.tensor([[30.0, 2.0],
                             [30.0, 2.0]])

target = torch.tensor([0, 1])  # Second sample belongs to class 1
# but logit for class 0 is greater: 30 > 2. So it was misclassified
# fmt: on

Подсчитаем Cross-Entropy Loss без весов:

$$\large \text{CE} = \frac{1}{2} \biggr[- \log\frac{e^{30}}{e^{30}+e^{2}} - \log\frac{e^{2}}{e^{30}+e^{2}}\biggr]\approx 14.0 $$

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
loss = criterion(model_output, target)
print(f"Loss = {loss.item():.2f}")

Если у нас есть два класса с соотношением $4:1$, можно задать веса `weight = [0.2, 0.8]`. И, так как сеть ошиблась на классе с большим весом, ошибка вырастет:

$$\large \text{CE}_{\text{weighted}} =  \biggr[\mathbf{-0.2} \log\frac{e^{30}}{e^{30}+e^{2}} -\mathbf{0.8} \log\frac{e^{2}}{e^{30}+e^{2}}\biggr]\approx 22.4 $$

In [ ]:
weights = torch.tensor([0.2, 0.8])
criterion = torch.nn.CrossEntropyLoss(weight=weights)
loss = criterion(model_output, target)
print(f"Loss = {loss.item():.2f}")

Сумма весов не обязана быть равной единице, важно только соотношение между весами. Часто соотношение весов делают обратно пропорциональным количеству объектов в классах.

In [ ]:
criterion = torch.nn.CrossEntropyLoss(weight=torch.tensor([1.0, 4.0]))
loss = criterion(model_output, target)
print(f"Loss = {loss.item():.2f}")

- Иногда качество модели можно улучшить, взяв квадратные корни от полученных таким образом весов (немного снижает штрафы за ошибки на редких классах).

- Несмотря на интуитивно понятную логику работы способа, он не всегда дает значительный эффект. Тем не менее, на практике стоит пробовать экспериментировать с этим способом наряду с прочими техниками борьбы с дисбалансом.

### Negative Log Likelihood


Еще одной функцией потерь, которая может использоваться при решении задач классификации наряду с Cross-Entropy Loss и может встречаться при работе в PyTorch, является обратный логарифм правдоподобия (Negative Log Likelihood Loss, NLL Loss).

Данная функция потерь отличается от Cross-Entropy Loss тем, что в качестве выхода модели она ожидает **не логиты**, а **логарифмы вероятностей для классов**.

Для $i$-го объекта выборки, если выходной вектор состоит из $C$ компонент (**логарифмов вероятностей** для $C$ классов), обратный логарифм правдоподобия между выходом модели $\hat{y}$ и целевым вектором $y$ будет равен:

$$\large \text{NLL}_i(\hat{y},y)= - \sum_{k=1}^{C}{y_{ik}\cdot\hat{y}_{ik}}$$

При вычислении по всему набору данных (или по мини-батчу) из $N$ объектов ошибка на отдельных объектах усредняется:

$$\large \text{NLL}=\frac{1}{N}\sum_{i=1}^{N}\text{NLL}_i$$


Для того, чтобы пользоваться NLL Loss при решении задачи классификации, к логитам, которые выдает модель, необходимо дополнительно применять Softmax и брать от результата натуральный логарифм, и уже результат такого вычисления передавать в NLL Loss. В PyTorch вычисление логарифма от результата применения Softmax к логитам реализовано в модуле `LogSoftmax` [🛠️[doc]](https://pytorch.org/docs/stable/generated/torch.nn.LogSoftmax.html). Взаимоотношение между NLL Loss и Cross-Entropy Loss можно выразить следующей иллюстрацией:

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/ce_loss_vs_nll_loss.png" width="900"></center>

In [ ]:
criterion = nn.NLLLoss()
logsoftmax = nn.LogSoftmax(dim=1)

print(f"model_output:\n {model_output}")

logprobs = logsoftmax(model_output)
print(f"logprobs:\n {logprobs}")

print(f"target: {target}")

loss_nll = criterion(logprobs, target)
print(f"loss_nll: {loss_nll}")

Двойственность NLL Loss и Cross-Entropy Loss может немного путать. Она возникла из-за того, что NLL Loss была исторически раньше реализована в библиотеке. Было принято на выходе модели ставить LogSoftmax, и использовать NLL Loss. Позднее была реализована функция Cross-Entropy Loss, которая включала в себя одновременно LogSoftmax и NLL Loss и позволяла не добавлять в модель лишний модуль. За счет такого упрощения использование Cross-Entropy Loss быстро стало более популярно. Реализацию NLL Loss, по всей видимости, оставили в библиотеке скорее для обратной совместимости.

Еще одним важным вытекающим отличием в использовании NLL Loss и Cross-Entropy Loss является следующее. После обучения модели мы хотим пользоваться ей по назначению — для классификации. Иногда нам хочется, чтобы мы могли смотреть на выходы модели как на вероятности отнесения объектов к различным классам.

* При использовании связки LogSoftmax + NLL Loss на выходе модели имеем логарифмы от вероятностей, и для получения самих вероятностей мы должны взять экспоненту от выхода модели.

* При использовании Cross-Entropy Loss на выходе модели имеем логиты, и для получения вероятностей мы должны применить Softmax-преобразование.

[[blog] ✏️ Объяснение Negative Log Likelihood Loss](https://ljvmiranda921.github.io/notebook/2017/08/13/softmax-and-the-negative-log-likelihood/)

[[blog] ✏️ О соотношении Cross-Entropy Loss и Negative Log Likelihood Loss](https://jamesmccaffrey.wordpress.com/2020/06/11/pytorch-crossentropyloss-vs-nllloss-cross-entropy-loss-vs-negative-log-likelihood-loss/)

###  Binary Cross-Entropy

В частном случае, когда количество классов равно двум (**задача бинарной классификации**), их можно закодировать одним числом: $0$ — для первого класса, и $1$ — для второго. Тогда сумму $\displaystyle \sum_{k=1}^{C}$ в формуле Cross-Entropy Loss можно расписать в явном виде.

Для $i$-го объекта выборки, когда выход модели является скаляром (**вероятностью** отнесения объекта к классу $1$), бинарная кросс-энтропия между выходом модели $\hat{y}$  и целевым значением $y$ будет равна

$$\text{BCE}_i(\hat{y},y)= - [{y_i\cdot\log(\hat{y_i})+(1-y_i)\cdot\log(1-\hat{y_i})}]$$

При вычислении по всему набору данных (или по мини-батчу) из $N$ объектов ошибка на отдельных объектах усредняется:

$$\text{BCE}=\frac{1}{N}\sum_{i=1}^{N}\text{BCE}_i$$

[[doc] 🛠️ Binary Cross-Entropy Loss в PyTorch](https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html#torch.nn.BCELoss):

```python
torch.nn.BCELoss()
```



Важной особенностью BCE Loss является то, что здесь используется не one-hot кодирование целевых значений для двух классов, а **одно число: 0 — первый класс, 1 — второй класс.** При этом значения целевой переменной должны быть представлены как вещественные (float) числа.

In [ ]:
criterion = nn.BCELoss()

model_output = torch.rand(1)
print(f"model_output: {model_output}")

target = torch.empty(1).random_(2)
print(f"target: {target}")

loss_bce = criterion(model_output, target)
print(f"loss_bce: {loss_bce}")

### Binary Cross-Entropy With Logits

Для того, чтобы выход модели при бинарной классификации представлял собой **вероятность отнесения объекта к классу $1$**, на выходе модели мы должны использовать логистическую функцию (sigmoid), результат которой можно будет передать в BCE Loss.

По аналогии с NLL Loss и Cross-Entropy Loss, у BCE Loss есть своя "пара": BCE With Logits Loss. Эта функция потерь совмещает в себе две операции:

* применение логистической функции Sigmoid,
* расчет BCE Loss.

Как можно понять из названия, BCE With Logits Loss на вход ожидает логиты. Взаимоотношение между BCE Loss и BCE With Logits Loss можно отобразить такой иллюстрацией:

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-content/L03/out/bce_loss_vs_bce_with_logits_loss.png" width="900"></center>

Функцию потерь BCE Loss (или ее "аналог" BCE With Logits Loss) можно применять не только в случае бинарной классификации, но и **в случае Multi-label классификации**.

В таком случае в векторе целевых значений вероятностей для каждого объекта может быть больше, чем одна единица. А логиты на последнем слое сети независимо пропускаются через сигмоиду (а не через Softmax, как в случае с многоклассовой классификацией).

#  Метод обратного распространения ошибки

Для эффективного численного расчета градиента функции потерь по обучаемым параметрам модели применяется **метод обратного распространения ошибки (backpropagation)**. Благодаря данному методу становится практически возможным использование метода градиентного спуска для проведения процедуры обучения.

Метод обратного распространения ошибки использует структуру многослойной нейронной сети как сложной функции, применяя правило дифференцирования сложной функции для вычисления градиента от функции потерь по весам сети. Градиент от функции потерь вычисляется при движении по нейронной сети от её выходов в направлении входов. Именно такой порядок обхода вычислительного графа и обуславливает название метода.

## Вычислительный граф

По существу, нейронная сеть является сложной функцией, работу которой можно представить как последовательное выполнение математических операций. Такое представление функций называется [вычислительным графом ✏️[blog]](https://qudata.com/ml/ru/ML_Comp_Graph.html).

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L05/out/nn_fully_connected.png"  width="500"></center>

Алгоритм обратного распространения ошибки позволяет находить градиенты для любого графа вычислений, если описываемая им функция дифференцируема.

В его основе лежит правило взятия производной сложной функции (chain rule):

Одна переменная:

$$\large y(x) = f(u(g(x))) $$

$$\large \frac{dy}{dx} = \frac{df}{du} \frac{du}{dg} \frac{dg}{dx}$$

Несколько переменных:

$$\large y(x) = f(u_1(x),u_2(x),...u_n(x))$$

$$\large \frac{dy}{dx} = \sum_{i=1}^{n} \frac{\partial f(u_1, u_2, ... u_n)}{\partial u_i} \frac{du_i}{dx}$$

$$\large \underbrace{\frac{d}{dx} f(\vec{\mathbf{u}}(x))}_{\text{Derivative of composition function}} = \overbrace{\nabla_{\vec{u}} f \cdot \vec{\mathbf{u}}'(x)}^{\text{Dot product of vectors}}$$

##  Пошаговый разбор метода обратного распространения

Пусть в каком-то узле графа производится вычисление

$$\large z = f(x, y),$$

и далее результат вычисления $\large z$ используется для вычисления функции $\large L(z)=L(f(x, y))$.

Тогда правило вычисления производных $\dfrac{\partial L}{\partial x}$ и $\dfrac{\partial L}{\partial y}$ можно представить следующим образом:

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L05/out/rule_for_taking_gradients.png"  width="500"></center>

Рассмотрим следующую функцию:

$$\Large f(w,x)=\frac{1}{1+e^{-(w_0x_0+w_1x_1+w_2)}}$$

Представим ее в виде вычислительного графа, состоящего из элементарных операций, от которых просто берутся производные:

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L05/out/graph_of_calculation_gradient.png"  width="700"></center>

На примере данной функции рассмотрим алгоритм обратного распространения ошибки и найдём величину её градиента по параметрам $\large w$.
Нам потребуется вычислить частные производные $\dfrac{\partial f}{\partial w_0}, \dfrac{\partial f}{\partial w_1}, \dfrac{\partial f}{dw_2}, \dfrac{\partial f}{\partial x_0}$ и $\dfrac{\partial f}{\partial x_1}$.

Пусть "веса" $w$ инициализированы значениями $w_0=2,\;w_1=-3,\;w_2=-3$, а "входные признаки" $x$ принимают значения $x_0=-1,\;x_1=-2$.

Делая прямой проход через граф вычислений для данной функции, получаем её значение для заданных $w$ и $x$ равным $f=0.73$:

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L05/out/forward_pass_example.png" width="800"></center>

Далее, в соответствии с алгоритмом обратного распространения ошибки, рассчитаем частные производные, пройдясь последовательно по графу вычислений, постепенно накапливая искомое значение для градиента функции.

Для начала зададим $\dfrac{df}{df}=1$.

Начинаем обратный проход по графу вычислений — первая вершина содержит функцию $f(x)=\dfrac{1}{x}$, производная которой равна $\dfrac{df}{dx}=-\dfrac{1}{x^2}$

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L05/out/compute_gradient_1_step.png" width="800"></center>

$$\large f(x)=\frac1x \quad \longrightarrow \quad \frac{df}{dx} = -\frac{1}{x^2}$$

В следующем узле находится функция $f(x)=1+x$. Производная от выражения в данном узле равняется $\dfrac{df}{dx}=1$:

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L05/out/compute_gradient_2_step.png" width="800"></center>

$$\large f(x)=c+x \quad \longrightarrow \quad \frac{df}{dx} = 1$$

Третья вершина содержит экспоненту $f(x)=e^x$. Её производная также является экспонентой $\dfrac{df}{dx}=e^x$:

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L05/out/compute_gradient_3_step.png" width="800"></center>

$$\large f(x)=e^x \quad \longrightarrow \quad \frac{df}{dx} = e^x$$

Следующая вершина, четвертая, содержит умножение на константу $f(x)=ax$. Производная равна $\dfrac{df}{dx}=a$ (в данном случае $a=-1$):

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L05/out/compute_gradient_4_step.png" width="800"></center>

$$\large f(x)=ax \quad \longrightarrow \quad \frac{df}{dx} = a$$

Двигаясь по графу вычислений, мы дошли до узла суммирования, который имеет два входа. Относительно каждого из входов локальный градиент в вершине суммирования будет равен $1$:
$$\large f(x,y)=x+y \quad \Rightarrow \quad \frac{\partial f}{\partial x}=1  \quad \quad \frac{\partial f}{\partial y}=1$$
Так как умножение на единицу не изменит значения входного градиента, всем входам узла суммирования мы можем приписать точно такое же значение входного градиента ($0.2$), что мы имели и для самого узла суммирования. Будем действовать аналогично и со всеми остальными узлами суммирования, которые встретятся нам в вычислительном графе.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L05/out/compute_gradient_5_step.png" width="800"></center>

Двигаясь далее к началу графа вычислений, мы подходим к вершинам умножения. Для такой вершины локальный градиент по отношению к какому-либо из входов будет равен значению оставшегося входа. Остается умножить локальный градиент на входящий.

$$\large f(w,x)=wx \quad \Rightarrow \quad \frac{\partial f}{\partial w}=x  \quad \quad \frac{\partial f}{\partial x}=w$$

Точно так же мы можем поступить и с оставшейся второй вершиной умножения, которая привязана к $w_1$ и $x_1$:

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L05/out/compute_gradient_6_step.png" width="800"></center>

Так, двигаясь по графу вычислений в обратном направлении от выхода функции к входным аргументам, мы последовательно для каждого узла умножаем локальный градиент на входящий градиент, используя цепное правило дифференцирования сложной функции. В описанном примере мы полностью разбили граф вычислений на отдельные элементарные узлы. Разбиение вычислительного графа на элементарные узлы вовсе не обязательно — мы можем сгруппировать несколько вершин вместе, если они образуют дифференцируемую функцию, от которой "удобно" брать производную, и рассматривать их совместно.

В нашем примере мы можем заметить, что вычислительный граф можно свести к двум операциям: получению выражения $w_0x_0+w_1x_1+w_2$ и последующему вычислению от него сигмоидальной функции.

Функция сигмоиды:

$$\large \displaystyle \sigma(x) = \frac{1}{1+e^{-x}}.$$

Важно отметить, что сигмоида обладает важным свойством: её производная может быть выражена через саму сигмоидальную функцию:

$$\large \frac{d}{dx}\sigma(x) = \frac{d}{dx}(1+e^{-x})^{-1} = \frac{e^{-x}}{(1+e^{-x})^{2}} = \frac{1}{1+e^{-x}} \cdot \frac{1+e^{-x}-1}{1+e^{-x}} = \sigma(x)\cdot(1-\sigma(x))$$

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L05/out/compute_gradient_join_vertices_sigmoid_example.png" width="800"></center>

В коде без использования библиотек подсчёт градиентов можно записать как:

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L05/out/calculating_gradients_in_code.png" width="800"></center>

Для базовых операций можно выделить следующие шаблоны "разделения" градиента:

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L05/out/add_copy_mul_max_gates.png" width="700"></center>

##  Анимация работы метода обратного распространения ошибки



Таким образом, метод обратного распространения ошибки включает в себя следующие шаги:
* Forward pass (FP) — прямое распространение сигнала от входа к выходам (без которого не получить вычисленные значения в графе).
* Backward pass (BP) — расчёт величины градиента функции потерь по весам от выходного слоя ко входному.
* Обновление весов в зависимости от величины градиента. На анимации буквой $\eta$ обозначен learning rate.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.1/L05/backprop_animation.gif" width="600"></center>

<center><em>Source: <a href="https://robocraft.ru/algorithm/560">Принцип обучения многослойной нейронной сети с помощью алгоритма обратного распространения</a></em></center>

Для более детального рассмотрения принципов работы и обучения нейронных сетей рекомендуем:

[[video] 📺 Плей-лист от 3Blue1Brown](https://youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi&si=O3gq6RoIxoNS6iJf) [[озвучка на русском](https://youtube.com/playlist?list=PLZjXXN70PH5itkSPe6LTS-yPyl5soOovc&si=ElKVi98Ui4m2nJKq)].

## Автоматическое вычисление градиента в PyTorch

PyTorch умеет запоминать последовательность операций с тензорами и вычислять градиент.

In [ ]:
import torch

x = torch.tensor([-1.0, -2.0])  # x from above example

x = torch.cat([x, torch.tensor([1.0])])  # concatenate x with 1. for bias trick

W = torch.tensor([2.0, -3.0, -3.0], requires_grad=True)  # w from above example

print(f"W.grad = {W.grad} (before forward and backward pass grad is 'None')")

# forward pass to compute f
s = x.matmul(W)
f = torch.sigmoid(s)
print(f"f(x, W) = {f:.2f}")

# backward pass to compute gradient df/dW
f.backward()
print(f"W.grad = {W.grad}")

Получили такие же значения частных производных $\dfrac{\partial f}{\partial w_i}$, как и при вычислении вручную (на иллюстрации градиент расчитан с точностью до двух знаков после запятой, в коде выше получены более точные значения).

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L05/out/autograd_example.png" width="700"></center>

Мы не требовали возможности расчета градиента по аргументу `x` (не указали `requires_grad=True`), поэтому после вызова `f.backward()` градиент по нему не рассчитается:

In [ ]:
print(f"x.grad = {x.grad}")

Отсоединение тензора от графа вычислений производится с помощью `.detach()` (используется, если нужно производить какие-либо дальнейшие операции с тензором, но чтобы они не становились частью вычислительного графа):

In [ ]:
f_detached = f.detach()

print(f"f_detached = {f_detached:.2f}")
print(f"f_detached type: {type(f_detached)}")

Также от тензора, содержащего скаляр, можно получить его величину с помощью `.item()`:

In [ ]:
value = f_detached.item()

print(f"value = {value:.2f}")
print(f"value type: {type(value)}")

## Обратное распространение в PyTorch


Рассмотрим алгоритм обратного распространения  ошибки на примере вычисления бинарной кросс-энтропии для логистической регрессии (для простоты не будем рассматривать смещение):

$$\large \hat y = σ(z) = \frac{1}{1+e^{-z}} $$

$$\large z = x_0 * w_0 + x_1 * w_1 = W^T X$$

Для примера возьмем следующие значения:
$$\large x=[[0, 0.8], [-0.1, 1.1], [-0.2, 0.8], [0.2, 0.7]],$$
$$\large y=[0, 1, 1, 0],$$
$$\large w=[[-5.1], [0.4]], b=0.5$$

Предсказания модели $\hat{y}=[0.58, 0.80, 0.90, 0.18]$ не совпадают с истинными значениями $y$. Соответственно, значение функции бинарной кросс-энтропии для такого примера будет:

$$\large \text{BCE}=-\frac{1}{4}\sum^{4}_{i=1}[y_ilog(\hat y_i)+(1-y_i)log(1-\hat y_i)]=-\frac{-0.87-0.22-0.11-0.20}{4}=0.35$$

Градиент функции потерь по весу $W$ вычисляется следующим образом, в соответствии с цепным правилом:

$$\large \frac{\partial \text{BCE}}{\partial W} = \frac{\partial \text{BCE}}{\partial \hat y} \frac{\partial \hat y}{\partial z}\frac{\partial z}{\partial W}$$

Рассчитаем градиент с использованием PyTorch и изобразим на графике границу решений:

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

X = torch.tensor([[0, 0.8], [-0.1, 1.1], [-0.2, 0.8], [0.2, 0.7]])
y = torch.tensor([0, 1, 1, 0])

# This is the parameter we want to optimize -> requires_grad=True
W = torch.tensor([[-9.1], [0.4]], requires_grad=True)
print(f"W.grad = {W.grad} (before forward pass must be 'None')\n")

# forward pass to compute Binary Cross-Entropy Loss
y_pred = 1/(1 + torch.exp(-(torch.mm(X, W).squeeze())))
BCE = -torch.mean(y*(torch.log(y_pred)) + (1-y)*torch.log(1-y_pred))
print(f"BCE = {BCE:.2f}\n")

# backward pass to compute gradient dBCE/dw
BCE.backward()
print(f"W.grad = {W.grad}")

# draw a decision boundary
W = W.clone().detach()
x1 = torch.tensor([min(X[:,0]), max(X[:,0])])
a = -W[0]/W[1]
x2 = a*x1

fig = plt.figure(figsize=(4,2))
colors = ListedColormap(['green', 'blue'])
plt.scatter(X[:, 0], X[:, 1], marker="o", c=y, s=40, cmap=colors)
plt.xlim([-0.3, 0.3])
plt.ylim([0.6, 1.2])
plt.xlabel("Признак 1")
plt.ylabel("Признак 2")
plt.plot(x1, x2, 'y-')
plt.show()

В данном примере мы произвели следующие расчеты:

$\displaystyle \large \frac{\partial \text{BCE}}{\partial \hat y} = - (\frac{y}{\hat y} - \frac{1-y}{1-\hat y}) = \frac{\hat y - y}{\hat y (1 - \hat y)}$
$\displaystyle \large \frac{\partial \hat y}{\partial z} = \frac{\partial}{\partial z}\left[\frac{1}{1+e^{-z}}\right] = \frac{\partial}{\partial z} (1+e^{-z})^{-1} = -(1+e^{-z})^{-2}(-e^{-z}) = \frac{e^{-z}}{(1+e^{-z})^2} = \frac{1}{1+e^{-z}} \frac{e^{-z}}{1+e^{-z}} = \frac{1}{1+e^{-z}} \frac{(1+e^{-z})-1}{1+e^{-z}}= \frac{1}{1+e^{-z}} \left ( \frac{1+e^{-z}}{1+e^{-z}} - \frac{1}{1+e^{-z}} \right ) = \frac{1}{1+e^{-z}} \left ( 1 - \frac{1}{1+e^{-z}} \right )= \hat y (1-\hat y)$
$\displaystyle \large \frac{\partial z}{\partial W} = X$

$\displaystyle \large \frac{\partial \text{BCE}}{\partial W} = \frac{\hat y - y}{\hat y (1 - \hat y)}\hat y (1-\hat y)X = (\hat y - y)X = [0.0769, 0.2764]$

In [ ]:
torch_grad = torch.tensor([[0.0192], [0.0691]])
print(f'torch: W.grad = {torch_grad}')
custom_grad = torch.mm((y_pred-y).unsqueeze(0), X)/4
print(f'custom: W.grad = {custom_grad}')

`ВСE.backward()` автоматически вычисляет градиент $\dfrac{\partial \text{BCE}}{\partial W}$ при указании `requires_grad=True`.
Результаты вычислений будут храниться в `W.grad`.

Также после однократного обратного прохода в целях экономии памяти граф, используемый для вычисления градиента, будет удаляться, и следующий запуск `BCE.backward()` будет выдавать ошибку:



```
BCE.backward() # Error on second backward call
```



Чтобы сохранить вычислительный граф, нужно взывать `backward()` с параметром `retain_graph=True`. В таком случае значения градиентов, полученные на следующих итерациях обратного распространения ошибки, будут складываться с текущими значениями градиентов.

Градиенты переменных, для которых был указан `requires_grad=True`, сохраняются автоматически. Чтобы избежать их накопления при многократном итерировании алгоритма обратного распространения, нужно обнулять градиент на каждом шаге с помощью метода `Тensor.grad.zero_()`.

In [ ]:
X = torch.tensor([[0, 0.8], [-0.1, 1.1], [-0.2, 0.8], [0.2, 0.7]])
y = torch.tensor([0, 1, 1, 0])

# This is the parameter we want to optimize -> requires_grad=True
W = torch.tensor([[-9.1], [0.4]], requires_grad=True)

# forward pass to compute BCE
y_pred = 1/(1 + torch.exp(-(torch.mm(X, W).squeeze())))
BCE = -torch.mean(y*(torch.log(y_pred)) + (1-y)*torch.log(1-y_pred))

print("Backward 1:")
BCE.backward(retain_graph=True)
print(f"dBCE/dW = {W.grad} \n")

print("Without forward, backward 2. Gradient is accumulating:")
BCE.backward(retain_graph=True)
# Gradients are accumulated
print(f"dBCE/dW = {W.grad}\n")

print("Backward 3, but firstly nullify gradients:")
W.grad.zero_()  # Nullify gradients for W for the next iteration
BCE.backward(retain_graph=True)
print(f"dBCE/dW = {W.grad}")

Итак, мы умеем вычислять градиент $\displaystyle \frac{\partial \text{BCE}}{\partial W}$ для нашего примера. Теперь давайте с его помощью оптимизируем веса, используя алгоритм обратного распространения ошибки:

In [ ]:
X = torch.tensor([[0, 0.8], [-0.1, 1.1], [-0.2, 0.8], [0.2, 0.7]])
y = torch.tensor([0, 1, 1, 0])

W = torch.tensor([[-9.1], [0.4]], requires_grad=True)

# Define model output
def forward(X):
    return 1/(1 + torch.exp(-(torch.mm(X, W).squeeze())))

# Compute Cross-Entropy loss
def criterion(y_pred, y):
    return -torch.mean(y*(torch.log(y_pred)) + (1-y)*torch.log(1-y_pred))

print(f"Prediction before training: f(x) = {forward(X)}")
print(f"True values: y = {y}\n")

# Training
learning_rate = 0.5
num_epochs = 1000

for epoch in range(num_epochs):
    # Propagate forward
    y_pred = forward(X)

    # Compute Cross-Entropy loss
    CE = criterion(y_pred, y)

    # Propagate backward, compute gradients
    CE.backward()

    # Update weights
    with torch.no_grad():  #  We don't want this step to be the part of the computational graph
        W -= learning_rate * W.grad

    # Nullify gradients after updating to avoid their accumulation
    W.grad.zero_()

print(f"After training: W = {W}, loss = {CE.item():.8f}")

print(f"\nPrediction after training: f(x) = {forward(X)}")
print(f"True values: y = {y}")

# draw a decision boundary
W = W.clone().detach()
x1 = torch.tensor([min(X[:,0]), max(X[:,0])])
a = -W[0]/W[1]
x2 = a*x1

fig = plt.figure(figsize=(4,2))
plt.scatter(X[:, 0], X[:, 1], marker="o", c=y, s=40, cmap=colors)
plt.xlim([-0.3, 0.3])
plt.ylim([0.6, 1.2])
plt.xlabel("Признак 1")
plt.ylabel("Признак 2")
plt.title('Граница решений')
plt.plot(x1, x2, 'y-')
plt.show()

Видно, что наш подход позволяет оптимизировать вес $W$ регрессии из примера и таким образом добиться почти идеального предсказания нашей модели, однако в данном подходе дополнительно можно автоматизировать вычисление функции потерь и обновление параметров с учетом градиента, используя готовые функции потерь из `torch.nn` и оптимизаторы из `torch.optim`.

In [ ]:
import torch.nn as nn

X = torch.tensor([[0, 0.8], [-0.1, 1.1], [-0.2, 0.8], [0.2, 0.7]])
y = torch.tensor([0, 1, 1, 0], dtype=torch.float32)

W = torch.tensor([[-9.1], [0.4]], requires_grad=True)

# Define model output
def forward(X):
    return 1/(1 + torch.exp(-(torch.mm(X, W).squeeze())))

# Compute Cross-Entropy loss
def criterion(y_pred, y):
    return -torch.mean(y*(torch.log(y_pred)) + (1-y)*torch.log(1-y_pred))

print(f"Prediction before training: f(x) = {forward(X)}")
print(f"True values: y = {y}\n")

# Training
learning_rate = 0.5
num_epochs = 1000

criterion = nn.MSELoss()
optimizer = torch.optim.SGD([W], lr=learning_rate)

for epoch in range(num_epochs):
    # Propagate forward
    y_pred = forward(X)

    # Compute Cross-Entropy loss
    CE = criterion(y_pred, y)

    # Propagate backward, compute gradients
    CE.backward()

    # Update weights
    optimizer.step()

    # Nullify gradients after updating to avoid their accumulation
    optimizer.zero_grad()

print(f"After training: W = {W}, loss = {CE.item():.8f}")

print(f"\nPrediction after training: f(x) = {forward(X)}")
print(f"True values: y = {y}")

# draw a decision boundary
W = W.clone().detach()
x1 = torch.tensor([min(X[:,0]), max(X[:,0])])
a = -W[0]/W[1]
x2 = a*x1

fig = plt.figure(figsize=(4,2))
plt.scatter(X[:, 0], X[:, 1], marker="o", c=y, s=40, cmap=colors)
plt.xlim([-0.3, 0.3])
plt.ylim([0.6, 1.2])
plt.xlabel("Признак 1")
plt.ylabel("Признак 2")
plt.title('Граница решений')
plt.plot(x1, x2, 'y-')
plt.show()